# 🏦 Quant-X: 멀티에이전트 금융 Deep Research 시스템 구축 실습

> **Fastcampus 금융 도메인 특화 AI Agent 구축 과정**

이 노트북은 단일 에이전트부터 시작하여, 보안/권한 관리(Governance)가 포함된 엔터프라이즈급 멀티 에이전트 시스템을 단계별로 구축하는 전체 과정을 담고 있습니다.

### 📋 실습 목차
1. **환경 설정**: 라이브러리 설치 및 디렉토리 구조 생성
2. **싱글 에이전트**: 기본 구조 및 로깅 시스템
3. **멀티 에이전트**: Manager-Worker 패턴 및 정보 수집
4. **도구(Tools) & RAG**: 사내 지식 검색 및 웹 검색 연동
5. **가드레일 & 보안**: 입력 필터링 및 규제 준수(Compliance)
6. **권한 관리**: 사용자 역할(RBAC) 및 감사 로그
7. **최종 배포**: 통합 시나리오 테스트

## 0. 환경 설정 (Environment Setup)

In [ ]:
# [Step 1] 필수 라이브러리 설치
!pip install smolagents langchain langchain-community langchain-openai faiss-cpu yfinance duckduckgo-search openai tiktoken python-dotenv plotly

In [ ]:
# [Step 2] 프로젝트 디렉토리 구조 생성
import os

base_dirs = [
    "QuantX_Project/core",
    "QuantX_Project/agents",
    "QuantX_Project/data/reports",
    "QuantX_Project/data/vector_store"
]

for dir_path in base_dirs:
    os.makedirs(dir_path, exist_ok=True)
    print(f"✅ 디렉토리 생성: {dir_path}")

In [ ]:
# [Step 3] API 키 설정
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key를 입력하세요: ")
    print("API Key 설정 완료")

## 싱글 에이전트 역할 설계 및 로그 시스템
금융 시스템의 핵심인 **감사 로그(Audit Logging)** 모듈을 먼저 구현하고, 이를 사용하는 기본 싱글 에이전트를 작성합니다.

In [ ]:
%%writefile QuantX_Project/core/logger.py
"""
[Audit Logging] 감사 로그 시스템
"""
import logging
import os
import json
from datetime import datetime
from typing import List, Dict, Any, Optional
from pathlib import Path
from collections import deque
import threading

class AuditLogger:
    def __init__(self, log_file_path: str = "./data/system.log"):
        self.log_file_path = Path(log_file_path)
        self.log_file_path.parent.mkdir(parents=True, exist_ok=True)
        self.lock = threading.Lock()
        self.memory_buffer = deque(maxlen=100)
        self._setup_logger()
        self.log_system_event("SYSTEM_START", "감사 로그 시스템 초기화 완료")
    
    def _setup_logger(self):
        self.logger = logging.getLogger("QuantX_Audit")
        self.logger.setLevel(logging.INFO)
        for handler in self.logger.handlers[:]:
            self.logger.removeHandler(handler)
        
        file_handler = logging.FileHandler(self.log_file_path, mode='a', encoding='utf-8')
        file_handler.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
        file_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)
        
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        console_handler.setFormatter(formatter)
        self.logger.addHandler(console_handler)
    
    def log_audit(self, user_id: str, action: str, details: Dict[str, Any] = None):
        with self.lock:
            log_message = f"USER:{user_id} | ACTION:{action}"
            if details:
                log_message += f" | DETAILS:{json.dumps(details, ensure_ascii=False)}"
            self.logger.info(log_message)
            self.memory_buffer.append({"timestamp": datetime.now().isoformat(), "message": log_message})
    
    def log_system_event(self, event_type: str, message: str, details: Dict[str, Any] = None):
        with self.lock:
            log_message = f"SYSTEM | {event_type} | {message}"
            self.logger.info(log_message)

    def log_security_event(self, user_id: str, event_type: str, message: str, severity: str = "WARNING", details: Dict[str, Any] = None):
        with self.lock:
            log_message = f"SECURITY | {severity} | USER:{user_id} | {event_type} | {message}"
            if severity == "CRITICAL": self.logger.critical(log_message)
            elif severity == "WARNING": self.logger.warning(log_message)
            else: self.logger.info(log_message)

# 전역 인스턴스
audit_logger = AuditLogger()

def get_logger(name: str) -> logging.Logger:
    logger = logging.getLogger(name)
    if not logger.handlers:
        handler = logging.StreamHandler()
        logger.addHandler(handler)
        logger.setLevel(logging.INFO)
    return logger

In [ ]:
%%writefile QuantX_Project/agents/single_core.py
"""
[Single Agent Core] 초기 단일 에이전트 구조
"""
import os
import logging
from smolagents import CodeAgent, LiteLLMModel
from core.logger import get_logger

logger = get_logger(__name__)

class QuantXAgent:
    def __init__(self, user_id: str):
        self.user_id = user_id
        self.is_demo_mode = not os.getenv('OPENAI_API_KEY')
        if not self.is_demo_mode:
            self.model = LiteLLMModel(model_id="gpt-4o")
            self.agent = CodeAgent(tools=[], model=self.model, add_base_tools=True)
    
    def process_request(self, request: str) -> str:
        if self.is_demo_mode:
            return f"[데모 모드] {request} 처리 완료"
        return str(self.agent.run(request))

def create_agent(user_id: str) -> QuantXAgent:
    return QuantXAgent(user_id)

In [ ]:
# [실습] 싱글 에이전트 테스트
import sys
import os
sys.path.append(os.path.abspath('./QuantX_Project'))

from agents.single_core import create_agent

print("🤖 싱글 에이전트 테스트...")
agent = create_agent("student01")
result = agent.process_request("Python으로 1부터 10까지 더하는 코드를 짜줘")
print(f"\n[결과]: {result}")

## 멀티 에이전트와 정보 수집 파이프라인
**Manager-Worker 패턴**을 도입하여 리서치(Research), 분석(Analyst), 관리(Manager) 역할을 분리합니다.

In [ ]:
%%writefile QuantX_Project/agents/core.py
"""
[Multi-Agent Core] Quant-X 멀티 에이전트 협업 시스템
"""
import os
import logging
from datetime import datetime
from typing import Dict, Any, List
from smolagents import CodeAgent, LiteLLMModel
from core.logger import get_logger
from agents.tools import search_internal, search_web, get_stock_price, get_market_summary, save_report

logger = get_logger(__name__)

# 세션 상태 (에이전트 간 메모리 공유)
AGENT_SESSION_STATE = {
    "research_results": [],
    "collaboration_log": []
}

class ResearchAgent:
    """정보 수집 전문가"""
    def __init__(self, user_id: str):
        self.user_id = user_id
        self.is_demo_mode = not os.getenv('OPENAI_API_KEY')
        if not self.is_demo_mode:
            model = LiteLLMModel(model_id="gpt-4o")
            self.agent = CodeAgent(tools=[search_internal, search_web], model=model, add_base_tools=True)

    def research(self, query: str) -> str:
        logger.info(f"[ResearchAgent] 정보 수집: {query}")
        AGENT_SESSION_STATE["collaboration_log"].append({"agent": "ResearchAgent", "action": f"정보수집: {query}"})
        if self.is_demo_mode: return f"[데모] {query} 정보 수집 완료"
        return str(self.agent.run(f"금융 리서치 전문가로서 다음을 조사해줘: {query}"))

class MarketAnalystAgent:
    """시장 데이터 분석가"""
    def __init__(self, user_id: str):
        self.user_id = user_id
        self.is_demo_mode = not os.getenv('OPENAI_API_KEY')
        if not self.is_demo_mode:
            model = LiteLLMModel(model_id="gpt-4o")
            self.agent = CodeAgent(tools=[get_stock_price, get_market_summary], model=model, add_base_tools=True)

    def analyze(self, query: str) -> str:
        logger.info(f"[MarketAnalyst] 시장 분석: {query}")
        AGENT_SESSION_STATE["collaboration_log"].append({"agent": "MarketAnalystAgent", "action": f"시장분석: {query}"})
        if self.is_demo_mode: return f"[데모] {query} 분석 완료"
        return str(self.agent.run(f"시장 애널리스트로서 수치와 데이터를 기반으로 분석해줘: {query}"))

class ManagerAgent:
    """프로젝트 매니저"""
    def __init__(self, user_id: str):
        self.user_id = user_id
        self.researcher = ResearchAgent(user_id)
        self.analyst = MarketAnalystAgent(user_id)

    def manage_request(self, request: str) -> str:
        logger.info(f"[Manager] 요청 관리: {request}")
        
        # 1. 리서치 실행
        research_res = self.researcher.research(request)
        # 2. 분석 실행
        analysis_res = self.analyst.analyze(request)
        
        # 3. 종합 (단순 문자열 결합 혹은 LLM을 통한 요약 가능)
        final_report = f"# 종합 보고서\n\n## 1. 리서치 결과\n{research_res}\n\n## 2. 시장 분석\n{analysis_res}"
        return final_report

class QuantXMultiAgent:
    def __init__(self, user_id: str):
        self.manager = ManagerAgent(user_id)
    
    def process_request(self, request: str) -> str:
        return self.manager.manage_request(request)
    
    def get_collaboration_log(self):
        return AGENT_SESSION_STATE["collaboration_log"]

## Tool 호출과 RAG 엔진 구현
에이전트가 사용할 **도구(Tools)**와 사내 지식 검색을 위한 **RAG 엔진**을 구현합니다. 금융 데이터의 정확성을 위해 RAG를 우선 사용합니다.

In [ ]:
%%writefile QuantX_Project/core/rag_engine.py
"""
[Knowledge Management] RAG 엔진 - 사내 금융 지식 검색 시스템
"""
import os
import logging
from pathlib import Path
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

class RAGEngine:
    _instance = None
    _initialized = False
    
    def __new__(cls):
        if cls._instance is None: cls._instance = super(RAGEngine, cls).__new__(cls)
        return cls._instance
    
    def __init__(self):
        if not self._initialized:
            self.vector_store = None
            self.logger = logging.getLogger(__name__)
            self._setup_directories()
            RAGEngine._initialized = True
    
    def _setup_directories(self):
        self.data_dir = Path("./data")
        (self.data_dir / "vector_store").mkdir(parents=True, exist_ok=True)
    
    def initialize(self):
        # 교육용 더미 데이터 생성
        self.logger.info("RAG 엔진 초기화 (더미 데이터)")
        if not os.getenv("OPENAI_API_KEY"): return
        
        dummy_data = [
            "삼성전자는 2024년 AI 반도체 점유율 확대를 목표로 하고 있다.",
            "미국 연준은 금리 인하 시기를 신중하게 결정할 것이라고 발표했다.",
            "이차전지 시장은 전기차 수요 둔화로 인해 일시적 조정을 받고 있다."
        ]
        docs = [Document(page_content=text, metadata={"source": "사내리포트"}) for text in dummy_data]
        embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
        self.vector_store = FAISS.from_documents(docs, embeddings)
    
    def search(self, query: str, k: int = 3) -> list:
        if not self.vector_store:
            return [{"content": "(RAG 미초기화 상태) 데이터가 없습니다.", "source": "System"}]
        
        docs = self.vector_store.similarity_search(query, k=k)
        return [{"content": doc.page_content, "source": doc.metadata.get("source")} for doc in docs]

rag_engine = RAGEngine()

In [ ]:
%%writefile QuantX_Project/agents/tools.py
"""
[AI Agent Tools] 금융 리서치를 위한 도구 함수들
"""
import logging
from functools import wraps
from datetime import datetime
import yfinance as yf
from duckduckgo_search import DDGS
from smolagents import tool
from core.logger import get_logger
from core.rag_engine import rag_engine
from core.auth import auth_manager

logger = get_logger(__name__)

# 미들웨어: 모든 도구 호출 시 감사 로그 기록
def middleware_wrapper(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tool_name = func.__name__
        logger.info(f"[Tool Execution] {tool_name} 실행")
        try:
            return func(*args, **kwargs)
        except Exception as e:
            logger.error(f"[Tool Execution] {tool_name} 실패: {e}")
            return f"오류 발생: {e}"
    return wrapper

@tool
@middleware_wrapper
def search_internal(query: str) -> str:
    """
    사내 지식베이스(RAG)를 검색합니다. 외부 검색보다 우선 사용해야 합니다.
    Args:
        query: 검색할 질문
    """
    results = rag_engine.search(query)
    formatted = "[사내 검색 결과]\n"
    for res in results:
        formatted += f"- {res['content']} (출처: {res['source']})\n"
    return formatted

@tool
@middleware_wrapper
def search_web(query: str) -> str:
    """DuckDuckGo를 사용하여 웹 검색을 수행합니다."""
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=3, region='kr-ko'))
    return str(results)

@tool
@middleware_wrapper
def get_stock_price(symbol: str) -> str:
    """Yahoo Finance를 통해 주가를 조회합니다. (예: 005930.KS)"""
    stock = yf.Ticker(symbol)
    hist = stock.history(period="1d")
    if hist.empty: return "주가 정보를 찾을 수 없습니다."
    price = hist['Close'].iloc[-1]
    return f"{symbol} 현재가: {price:,.0f}원"

@tool
@middleware_wrapper
def get_market_summary() -> str:
    """주요 시장 지수(코스피, 코스닥 등)를 요약합니다."""
    return "[시장 요약] 코스피: 2,600 (보합), 코스닥: 850 (상승) - 데모 데이터"

@tool
@middleware_wrapper
def save_report(title: str, content: str) -> str:
    """리포트를 파일로 저장합니다. (권한 필요)"""
    if not auth_manager.check_permission('save_report'):
        return "❌ 권한 부족: 리포트를 저장할 수 없습니다."
    
    filename = f"data/reports/{title}.md"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)
    return f"✅ 리포트 저장 완료: {filename}"


## 가드레일 & 보안 (Governance)
입력값의 유해성을 판단하고, 출력값에서 **규제 위반(Compliance)** 사항(예: 수익 보장 단어)을 필터링합니다.

In [ ]:
%%writefile QuantX_Project/core/guardrails.py
"""
[Security Guardrails] 보안 가드레일 시스템
"""
import logging
from enum import Enum
from dataclasses import dataclass
from typing import Optional, List

class RiskLevel(Enum):
    SAFE = "safe"
    BLOCKED = "blocked"

@dataclass
class GuardrailResult:
    is_safe: bool
    risk_level: RiskLevel
    message: str
    filtered_content: Optional[str] = None

class SecurityGuardrails:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        self.input_blacklist = ["해킹", "sql injection", "drop table"]
        self.compliance_violations = ["무조건", "100% 보장", "확실한 수익"]
    
    def check_input(self, user_input: str) -> GuardrailResult:
        for keyword in self.input_blacklist:
            if keyword in user_input.lower():
                return GuardrailResult(False, RiskLevel.BLOCKED, f"차단 키워드 감지: {keyword}")
        return GuardrailResult(True, RiskLevel.SAFE, "안전함")

    def filter_output(self, output: str) -> GuardrailResult:
        filtered = output
        detected = []
        for word in self.compliance_violations:
            if word in filtered:
                filtered = filtered.replace(word, "[규제준수 대체어]")
                detected.append(word)
        
        if detected:
            return GuardrailResult(True, RiskLevel.SAFE, "필터링 적용됨", filtered_content=filtered)
        return GuardrailResult(True, RiskLevel.SAFE, "안전함", filtered_content=output)

security_guardrails = SecurityGuardrails()

In [ ]:
# [실습] 보안 가드레일 테스트
from core.guardrails import security_guardrails

# 1. 입력 차단 테스트
print("🛡️ 입력 보안 테스트:")
res_in = security_guardrails.check_input("시스템을 해킹하고 싶어")
print(f"결과: {res_in.risk_level}, 메시지: {res_in.message}")

# 2. 출력 필터링 테스트
print("\n🛡️ 출력 규제 준수 테스트:")
res_out = security_guardrails.filter_output("이 종목은 무조건 100% 보장 수익이 납니다.")
print(f"변환된 출력: {res_out.filtered_content}")

## 권한 관리 (Authentication)
사용자 등급(Junior/Senior)에 따라 수행 가능한 작업(리포트 저장 등)을 제어하는 **RBAC(Role-Based Access Control)** 시스템입니다.

In [ ]:
%%writefile QuantX_Project/core/auth.py
"""
[Authentication] 사용자 인증 및 권한 관리
"""
import logging
from dataclasses import dataclass
from typing import Dict, Optional

@dataclass
class UserSession:
    user_id: str
    role: str
    permissions: Dict[str, bool]

class AuthenticationManager:
    def __init__(self):
        self.current_session: Optional[UserSession] = None
        self.logger = logging.getLogger(__name__)
        self.roles = {
            "junior": {"save_report": False},
            "senior": {"save_report": True}
        }
    
    def login(self, user_id: str):
        role = "senior" if "senior" in user_id else "junior"
        self.current_session = UserSession(user_id, role, self.roles[role])
        self.logger.info(f"로그인 성공: {user_id} ({role})")
    
    def check_permission(self, permission: str) -> bool:
        if not self.current_session: return False
        return self.current_session.permissions.get(permission, False)

auth_manager = AuthenticationManager()

최종 배포 및 시나리오 테스트
이제 모든 모듈(에이전트, 도구, 보안, 권한)을 통합하여 **Manager Agent**가 전체 프로세스를 조율하는 과정을 실행합니다.

In [ ]:
# 통합 테스트 실행
import sys
import os
sys.path.append(os.path.abspath('./QuantX_Project'))

from agents.core import QuantXMultiAgent
from core.auth import auth_manager
from core.rag_engine import rag_engine

# 1. 시스템 초기화
rag_engine.initialize()

# 2. 로그인 (Senior 권한으로 리포트 저장 가능)
auth_manager.login("senior_manager_kim")

# 3. 멀티 에이전트 생성
system = QuantXMultiAgent("senior_manager_kim")

# 4. 복합 질문 요청
user_query = "삼성전자의 최근 주가와 반도체 시장 동향을 분석하고 리포트를 작성해줘"
print(f"🚀 요청 시작: {user_query}\n")

# 5. 실행 및 결과 출력
final_report = system.process_request(user_query)

print("\n================ [최종 산출물] ================")
print(final_report)
print("===============================================")

In [ ]:
# [감사 로그 확인]
# 에이전트들이 수행한 작업과 도구 호출 내역을 확인합니다.
print("📋 에이전트 협업 로그:")
logs = system.get_collaboration_log()
for log in logs:
    print(f"[{log.get('agent', 'System')}] {log.get('action')}")